# Scrapy
- https://scrapy.org/
- scrapy 기본 구조
- xpath
- Scrapy Project
- Scrapy Excute
- Pipelines

install scrapy
- `$ pip3 install scrapy`
- `conda install -c conda-forge scrapy `

### scrapy 기본 구조

- Spider
    - 어떤 웹사이트들을 어떻게 크롤링할 것인지 명시하고, 각각의 웹 페이지의 어떤 부분을 스크래핑할 것인지 명시하는 클래스

- Selector
    - 웹 페이지상의 특정 HTML Element를 간편하게 선택할 수 있도록 하는 메커니즘을 구현한 클래스
    - CSS Selector를 직접 사용하거나, XPath를 사용할 수 있음
    - XPath를 사용하는것이 더 권장됨

- items.py
    - 웹페이지에서 원하는 부분을 스크랩하여 저장할 때 사용하는 사용자 정의 자료구조 클래스

- pipeline.py
    - 스크래핑 결과물을 Item 형태로 구성하였을 때, 이를 자유롭게 가공하거나 다양한 파일 형태로 저장할 수 있도록 하는 클래스 (getter와 setter의 개념)

- settings.py
    - Spider나 Item Pipeline 등이 어떻게 동작하도록 할 지에 대한 세부적인 설정 사항을 기재하는 파일 크롤링의 빈도등을 설정
    - cf. robots.txt (settings.py - ROBOTSTXT_OBEY = True)

### xpath
- scrapy shell
- `$ scrapy shell "https://www.naver.com/"

In [4]:
import scrapy # Change here

In [5]:
import requests
from scrapy.http import TextResponse

In [6]:
# 웹 페이지에 연결
req = requests.get('https://www.naver.com/')

# response 객체 생성
response = TextResponse(req.url, body=req.text, encoding='utf-8')

xpath를 이용하여 특정 selector 객체 가져옴
- xpath : xpath 데이터
- data : xpath에서 선택된 위치의 데이터

- `//` : 가장 상위 Element
- `.` : 현재 Element
- `*` : 조건에 맞는 앞부분의 하위 Element를 모두 살펴봄 ( css selector에서 한칸 띄기와 같음 )
- `/` : 바로 아래 요소 ( css selector에서 > 와 같음 )
- `element[조건]`
    - p[2] : p element의 두번째 element : index가 0이 아닌 1부터 시작
    - [@(attribute key)="(attribute value)"]
    - [@id="email"] : 아이디 값이 email인 element
    - [@class="pw"] : 클래스 값이 pw인 element
- `not`
    - not(조건) : 조건이 아닌 요소를 찾음

In [7]:
response.xpath('//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li[1]/a/span[2]')

[<Selector xpath='//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li[1]/a/span[2]' data='<span class="ah_k">송하늘</span>'>]

In [8]:
# text
# xpath 가장 뒤에 text() 함수를 작성
response.xpath('//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li[1]/a/span[2]/text()')

[<Selector xpath='//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li[1]/a/span[2]/text()' data='송하늘'>]

In [9]:
# extract
# xpath로 select된 element의 데이터를 문자열 리스트로 가져옴
response.xpath('//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li[1]/a/span[2]/text()').extract()

['송하늘']

##### 여러개 selecte 해서 데이터 가져오기
- 네이버 실시간 키워드

In [10]:
naver_keywords = response.xpath('//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li')
len(naver_keywords)

20

In [11]:
naver_rank = naver_keywords.xpath('./a/span[1]/text()').extract()[:10]
naver_rank

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [12]:
naver_keywords_list = naver_keywords.xpath('./a/span[2]/text()').extract()[:10]
naver_keywords_list

['송하늘',
 '이영학',
 '어금니아빠',
 '방탄소년단 공식쇼핑몰',
 '오동식',
 '최다빈',
 '조민기 성추행',
 '박보검',
 '이영학 사형',
 '한국콜마']

### Scrapy Project
- g마켓 베스트 셀러 상품, 상품평
- http://corners.gmarket.co.kr/Bestsellers

- $ scrapy startproject crawler

```
.
├── gb_crawler
│   ├── __init__.py
│   ├── __pycache__
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       └── __pycache__
└── scrapy.cfg
```

In [13]:
req = requests.get('http://corners.gmarket.co.kr/Bestsellers')
response = TextResponse(req.url, body=req.text, encoding='utf-8')
bestsellers = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li')
len(bestsellers)

200

In [14]:
# titles
titles = bestsellers.xpath('./a/text()').extract()
len(titles), titles[0]

(200, '[다우니]다우니 퍼퓸 섬유유연제 2L 2개 + 200ml 4개')

In [15]:
# links
# links가 path로 되어 있을때, 전체 URL을 얻으려면 response.urljoin(path)로 실행
links = bestsellers.xpath('./a/@href').extract()
len(links), links[0]

(200,
 'http://item2.gmarket.co.kr/Item/DetailView/Item.aspx?goodscode=253378390')

In [16]:
bestsellers = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li')[:5]
for bestseller in bestsellers:
    link = bestseller.xpath('./a/@href').extract()
    print(link) 

['http://item2.gmarket.co.kr/Item/DetailView/Item.aspx?goodscode=253378390']
['http://item2.gmarket.co.kr/Item/DetailView/Item.aspx?goodscode=1163080944']
['http://item2.gmarket.co.kr/Item/DetailView/Item.aspx?goodscode=127195332']
['http://item2.gmarket.co.kr/Item/DetailView/Item.aspx?goodscode=433472486']
['http://item2.gmarket.co.kr/Item/DetailView/Item.aspx?goodscode=933865243']


세부 페이지 크롤링

In [17]:
req = requests.get('http://item.gmarket.co.kr/Item?goodscode=1131170115')
response = TextResponse(req.url, body=req.text, encoding='utf-8')

In [18]:
title = response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract().strip()
title

'[라크로스] 강경준의 겨울 패딩조끼/패딩점퍼/남녀공용/빅사이즈'

In [19]:
s_price = response.xpath('//*[@id="itemcase_basic"]/p/span/strong/text()')[0].extract().replace(",","")
s_price

'13900'

In [20]:
o_price = response.xpath('//*[@id="itemcase_basic"]/p/span/span/text()')[0].extract().replace(",","")
o_price

'44900'

In [21]:
discount_rate = str(round((1 - int(s_price) / int(o_price)) * 100, 2)) + "%"
discount_rate

'69.04%'

### Scrapy Excute
- `$ scrapy crawl GmarketBestsellers`
- 결과를 csv 파일로 저장
    - `$ scrapy crawl GmarketBestsellers -o GmarketBestsellers.csv`
    - 한꺼번에 페이지를 로드해서 가져옴으로 저장되는 순서가 일정하지 않음
    - 컬럼 데이터도 순서가 뒤죽박죽

### Pipelines
- 결과를 깔끔하게 정리해서 저장
- spider의 items 데이터가 pipelines에 정의된 대로 저장
- pipelines.py 파일에 코드 추가

```
import csv

class GbCrawlerPipeline(object):

    def __init__(self):
        self.csvwriter = csv.writer(open("GmarketBestsellers.csv", "w"))
        self.csvwriter.writerow(["title","o_price","s_price","discount_rate"])

    def process_item(self, item, spider):
        row = []
        row.append(item["title"])
        row.append(item["o_price"])
        row.append(item["s_price"])
        row.append(item["discount_rate"])
        self.csvwriter.writerow(row)
        return item
```

- settings.py 에 아래 코드 추가

```
ITEM_PIPELINES = {
   'gb_crawler.pipelines.GbCrawlerPipeline': 300,
}
```